In [ ]:
import feedparser
import time
import requests
import re

from tqdm import tqdm
from pprint import pprint
from bs4 import BeautifulSoup

In [ ]:
feed_urls = [
    "https://www.dealnews.com/c142/Electronics/?rss=1",
        # "https://www.dealnews.com/c39/Computers/?rss=1",
        # "https://www.dealnews.com/c238/Automotive/?rss=1",
        # "https://www.dealnews.com/f1912/Smart-Home/?rss=1",
        # "https://www.dealnews.com/c196/Home-Garden/?rss=1",
       ]

In [ ]:
def extract(html_snippet):
    soup = BeautifulSoup(html_snippet, 'html.parser')
    snippet_div = soup.find('div', class_='snippet summary')
    
    if snippet_div:
        description = snippet_div.get_text(strip=True)
        description = BeautifulSoup(description, 'html.parser').get_text()
        description = re.sub('<[^<]+?>', '', description)
        result = description.strip()
    else:
        result = html_snippet
    return result.replace('\n', ' ')

In [ ]:
def parse_entry(entry):
    parsed = {}
    parsed['title'] = entry['title']
    parsed['summary'] = extract(entry['summary'])
    parsed['url'] = entry['links'][0]['href']
    stuff = requests.get(parsed['url']).content
    soup = BeautifulSoup(stuff, 'html.parser')
    content = soup.find('div', class_='content-section').get_text()
    content = content.replace('\nmore', '').replace('\n', ' ').strip()
    if "Features" in content:
        details, features = content.split("Features")
    else:
        details = content
        features = ""
    # parsed['content'] = content
    parsed['details'] = details
    parsed['features'] = features.strip()
    
    return parsed

In [ ]:
def fetch(show_progress=False):
    deals = []
    feed_iter = tqdm(feed_urls) if show_progress else feed_urls
    for feed_url in feed_iter:
        feed = feedparser.parse(feed_url)
        pprint(feed.entries[0])
        for entry in feed.entries[:10]:
            deals.append(parse_entry(entry))
            time.sleep(0.5)
    return deals

In [ ]:
deals = fetch(False)

In [ ]:
print(len(deals))

In [ ]:
print(type(deals[0]))

In [ ]:
for deal in deals:
    pprint(deal)
    print()